# Perceptron Algorithm for Classification of Iris Dataset

In [384]:
# load the iris dataset
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

iris = load_iris()
X = iris.data
y = iris.target

Preprocess the data

In [385]:
# Preprocess the data
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Define the perceptron algorithm

In [386]:
# Define the perceptron algorithm
class MultiClassPerceptron:
    def __init__(self, input_dim, output_dim, lr=0.01, epochs=1000):
        self.W = np.random.randn(input_dim, output_dim)
        self.b = np.zeros((1, output_dim))
        self.lr = lr
        self.epochs = epochs

    def forward(self, X):
        # compute the logits
        logits = np.dot(X, self.W) + self.b
        return logits


    def backward(self, X, y):
        # compute the gradients
        m = X.shape[0]
        logits = self.forward(X)
        softmax_output = self.softmax(logits)
        grad_logits = softmax_output
        grad_logits[range(m), y] -= 1
        grad_logits /= m

        # update weights and biases
        dW = np.dot(X.T, grad_logits)
        db = np.sum(grad_logits, axis=0)
        self.W -= self.lr * dW
        self.b -= self.lr * db

    def softmax(self, X):
        exp_scores = np.exp(X)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        return probs

    def fit(self, X, y):
        for epoch in range(self.epochs):
            self.backward(X, y)

    def predict(self, X):
        logits = self.forward(X)
        y_pred = np.argmax(logits, axis=1)
        return y_pred

Train the model

In [387]:
# Train the model
p = MultiClassPerceptron(input_dim=X_train.shape[1], output_dim=3, lr=0.01, epochs=1000)
p.fit(X_train, y_train)
predictions_train = p.predict(X_train)
predictions = p.predict(X_test)

Evaluate the model

In [388]:
# evaluate train accuracy
print("Perceptron classification train accuracy", accuracy_score(y_train, predictions_train))
print("Perceptron classification accuracy", accuracy_score(y_test, predictions))

Perceptron classification train accuracy 0.8833333333333333
Perceptron classification accuracy 0.9


Non-linear feature transformation on the concrete compressive strength dataset

In [389]:
from itertools import combinations_with_replacement

def polynomial_features(X, degree):
    X_poly = X.copy()
    for d in range(2, degree + 1):
        combinations = combinations_with_replacement(range(X.shape[1]), d)
        for comb in combinations:
            X_new = np.prod(X[:, comb], axis=1)
            X_poly = np.column_stack((X_poly, X_new))
    return X_poly


In [390]:
# Non-linear feature transformation
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# load the concrete compressive strength dataset
df = pd.read_excel('Concrete_Data.xls')

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('Concrete compressive strength(MPa, megapascals) ', axis=1), df['Concrete compressive strength(MPa, megapascals) '], test_size=0.2, random_state=42)

# transform the features into second degree polynomial features
X_train_poly_custom = polynomial_features(X_train.values, degree=2)
X_test_poly_custom = polynomial_features(X_test.values, degree=2)

Train the linear regression model

In [391]:
# Train the model
lr_poly_custom = LinearRegression()
lr = LinearRegression()
# fit the model
lr_poly_custom.fit(X_train_poly_custom, y_train)
lr.fit(X_train, y_train)
# predict values from the polynomial transformed features
predictions_poly_custom_train = lr_poly_custom.predict(X_train_poly_custom)
predictions_poly_custom = lr_poly_custom.predict(X_test_poly_custom)
# predict values from the original features
predictions_train = lr.predict(X_train)
predictions = lr.predict(X_test)

# mean squared error
print("Mean squared error (train poly custom): {:.2f}".format(mean_squared_error(y_train, predictions_poly_custom_train)))
print("Mean squared error (test poly custom): {:.2f}".format(mean_squared_error(y_test, predictions_poly_custom)))
print("Mean squared error (train): {:.2f}".format(mean_squared_error(y_train, predictions_train)))
print("Mean squared error (test): {:.2f}".format(mean_squared_error(y_test, predictions)))

# coefficient of determination (R^2)
print("R^2 (train poly custom): {:.2f}".format(r2_score(y_train, predictions_poly_custom_train)))
print("R^2 (test poly custom): {:.2f}".format(r2_score(y_test, predictions_poly_custom)))
print("R^2 (train): {:.2f}".format(r2_score(y_train, predictions_train)))
print("R^2 (test): {:.2f}".format(r2_score(y_test, predictions)))

Mean squared error (train poly custom): 53.09
Mean squared error (test poly custom): 55.59
Mean squared error (train): 110.66
Mean squared error (test): 95.98
R^2 (train poly custom): 0.81
R^2 (test poly custom): 0.78
R^2 (train): 0.61
R^2 (test): 0.63


RBFs on the California Housing Prices dataset

In [392]:
# Implement the rbf_kernel() function
def rbf_kernel(X, centers, gamma):
    diff = X[:, np.newaxis, :] - centers[np.newaxis, :, :]
    norm = np.linalg.norm(diff, axis=2)
    rbf = np.exp(-gamma * np.power(norm, 2))
    return rbf

In [393]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the California Housing Prices dataset
data = fetch_california_housing()
X = data['data']
y = data['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# Choose the number of centroids and the RBF kernel width
num_centroids = 100
gamma = 0.1

# Randomly select the centroids from the training set
np.random.seed(42)
idx = np.random.choice(X_train_std.shape[0], num_centroids, replace=False)
centroids = X_train_std[idx]

# Compute the RBF features for the training and testing sets
rbf_train = rbf_kernel(X_train_std, centroids, gamma)
rbf_test = rbf_kernel(X_test_std, centroids, gamma)

# Fit a linear regression model on the original and RBF-transformed data
linreg_orig = LinearRegression().fit(X_train_std, y_train)
linreg_rbf = LinearRegression().fit(rbf_train, y_train)

# Evaluate the models on the testing set
y_pred_orig = linreg_orig.predict(X_test_std)
mse_orig = mean_squared_error(y_test, y_pred_orig)
r2_orig = r2_score(y_test, y_pred_orig)

y_pred_rbf = linreg_rbf.predict(rbf_test)
mse_rbf = mean_squared_error(y_test, y_pred_rbf)
r2_rbf = r2_score(y_test, y_pred_rbf)

# Print the results
print("Linear regression on original data:")
print("MSE:", mse_orig)
print("R^2:", r2_orig)

print("\nLinear regression on RBF-transformed data:")
print("MSE:", mse_rbf)
print("R^2:", r2_rbf)


Linear regression on original data:
MSE: 0.5558915986952438
R^2: 0.5757877060324512

Linear regression on RBF-transformed data:
MSE: 0.3710644691311594
R^2: 0.7168330839511811


# **(Bonus)** Multilayer Perceptron Algorithm for Regression of Concrete Compressive Strength Dataset

Download the Concrete Compressive Strength Dataset from the UCI Machine Learning Repository.

In [394]:
# Download the Concrete Compressive Strength Dataset from the UCI Machine Learning Repository.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import numpy as np

# Load the dataset
df = pd.read_excel('Concrete_Data.xls')

Preprocess the data

In [395]:
# Preprocess the data
X = df.drop('Concrete compressive strength(MPa, megapascals) ', axis=1)
y = df['Concrete compressive strength(MPa, megapascals) ']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

Define the multilayer perceptron algorithm

In [396]:
# Implement the functions in the MLP class
class MLP:
    def __init__(self, input_dim, hidden_dim, output_dim, lr=0.01, epochs=1000):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lr = lr
        self.epochs = epochs

        # Initialize weights and biases
        self.W1 = np.random.randn(self.input_dim, self.hidden_dim)
        self.b1 = np.zeros((1, self.hidden_dim))
        self.W2 = np.random.randn(self.hidden_dim, self.output_dim)
        self.b2 = np.zeros((1, self.output_dim))

    def forward_propagation(self, X):
        # Calculate the activations of the hidden layer
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = np.tanh(self.z1)

        # Calculate the activations of the output layer
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.z2

    def backward_propagation(self, X, y):
        # Calculate the gradient of the output layer
        delta2 = self.a2 - y

        # Calculate the gradient of the hidden layer
        delta1 = np.dot(delta2, self.W2.T) * (1 - np.power(self.a1, 2))

        # Calculate the gradients of the weights and biases
        dW2 = np.dot(self.a1.T, delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)
        dW1 = np.dot(X.T, delta1)
        db1 = np.sum(delta1, axis=0)

        # Update the weights and biases using the gradients
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1


    def fit(self, X, y):
        self.loss = []

        for epoch in range(self.epochs):
            # Forward propagation
            self.forward_propagation(X)

            # Backward propagation
            self.backward_propagation(X, y)

            # Calculate the mean squared error
            mse = mean_squared_error(y, self.a2)
            self.loss.append(mse)

    def predict(self, X):
        # Forward propagation to predict the output
        self.forward_propagation(X)
        return self.a2.flatten()

Train the model

In [397]:
# Create an instance of the MLP class
mlp = MLP(input_dim=X_train.shape[1], hidden_dim=10, output_dim=1, lr=0.01, epochs=1000)

# Train the model
mlp.fit(X_train, y_train)

ValueError: Data must be 1-dimensional

Evaluate the model

In [ ]:
# Evaluate the model
y_pred = mlp.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))

Mean Squared Error: 36.8911071801165


Compare the results with the linear regression model

In [ ]:
# Fit a linear regression model on the training data
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)
y_pred_lr = linear_regression.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
print("Mean Squared Error (Linear Regression):", mse_lr)

Mean Squared Error: 95.97548435337708
